# FIFEBATCH Tutorial Notebook
This notebook is meant as a demonstration of code used to extract data from the FIFEBATCH parquet files. The backend interaction with the parquet files is handled with the python module `duckdb`, which allows for SQL-style queries of the parquet data with a streaming-based SQL execution engine. This allows for the processing of large datasets that cannot fit in memory. 

## Required Imports
This notebook uses several standard libraries for loading and processing data. The plotting library used exclusively here is `matplotlib`, though users should feel free to plot with whatever plotting library they wish.

Each of the required libraries is present in `requirements.txt` and can be installed with one line:

```bash
pip install -r requirements.txt
```

In [1]:
import duckdb
import numpy as np
import matplotlib.pyplot as plt

## What is Parquet?

Parquet is a popular binary file format for storing large datasets. It is columnar in nature, which means that all data is stored by column and not by row. Practically, this means that the data belonging to a single field is stored in one consecutive region of memory for efficient access. This makes operations acting on an entire field very efficient. Additionally, the file format utilizes compression to conserve disk space.

The Parquet format is popular for large datasets and used across many disciplines for these reasons.

## What is DuckDB?

DuckDB serves as a frontend for interacting with Parquet (and other file formats) files through standard SQL queries. This has the advantage of being quite formulaic and immediately familiar to people of a variety of backgrounds. The DuckDB database engine allows users to define a query to filter data before it even enters the Python code, thus making for efficient memory usage.

## A simple DuckDB example

With DuckDB, you can query Parquet files like they are database tables (think SQL):

In [2]:
rel = duckdb.sql("SELECT User FROM '../data/*.parquet'")

Here, we create a "Relation" object called `rel` that essentially acts as a data access plan. It can be interpreted as "I want all entries from the `User` field contained in the files matching the pattern `../data/*.parquet`." It's important to note that at this stage *no data has actually been accessed.* This is simply a description of the data that we want.

A Relation is a plan for our query, and the user gets to decide how to materialize it:
* `.df()` - a Pandas DataFrame
* `.fetchnumpy()` - A NumPy array
* `.fetch_arrow_table()` - Arrow table
* `.fetch_record_batch()` - Arrow batches (for streaming)

If a Parquet file has many rows (e.g. millions), pulling it all into memory at once can overwhelm the available system memory. This would manifest as a sluggish response in your computer and eventually a crashed Jupyter kernel (if using Jupyter notebooks). For now, we can fetch the result as a NumPy array:

In [3]:
data = rel.fetchnumpy()
print(type(data))
print(data.keys())
print(type(data['User']))

<class 'dict'>
dict_keys(['User'])
<class 'numpy.ndarray'>


This `data` object is a dictionary containing as key/values the fields you selected (here `User`) and the corresponding NumPy array for each. We can retrieve the array containing the `User` field and produce a count of the top ten users:

In [4]:
users = data['User']
unique, counts = np.unique(users, return_counts=True)
top10 = np.argsort(counts)[-10:]
display(unique[top10])

array(['amakovec@fnal.gov', 'gm2pro@fnal.gov', 'laliaga@fnal.gov',
       'imawby@fnal.gov', 'gputnam@fnal.gov', 'novapro@fnal.gov',
       'ichong@fnal.gov', 'cmsgli@fnal.gov', 'icaruspro@fnal.gov',
       'uboonepro@fnal.gov'], dtype=object)